<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

## raw data를 pandas와 파이썬으로 조작해서 그래프 만들어보기

### 1. 데이터 시각화란?
- 데이터 분석 결과를 쉽게 이해할 수 있도록 시각적으로 표현하고 전달되는 과정
- 탐색적 데이터 분석, 데이터 처리, 데이터 예측 모든 경우, 결과를 알아보기 쉽게 하기 위해 데이터 시각화는 필수적임
- 다양한 시각화 기법 중, 가장 최신의 흥미로운 데이터 시각화 과정을 진행해보기로 함
  - https://app.flourish.studio
  - https://public.flourish.studio/visualisation/2897018/

### 지금까지 익힌 데이터 처리 기술을 기반으로 데이터 시각화를 위해, raw data를 포멧에 맞추어 변환하여 그래프를 만들어보기로 함
<img src="https://www.fun-coding.org/00_Images/covid_graph_ex2.jpg" />

### 2. 데이터 시각화를 위한 데이터 포멧 이해
- 데이터 시각화를 위해, raw data를 변환해야 함
- 지금까지 익힌 데이터 처리 기술을 사용해서, 데이터를 변환하기로 함

- 필요 데이터
  - 국가명, 국기, 날짜별 확진자 수

<img src="https://www.fun-coding.org/00_Images/covid_ex_data_format.jpg" />

### 3. raw data 가져오기

In [ ]:
import pandas as pd
PATH = "COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"
doc = pd.read_csv(PATH + "04-01-2020.csv", encoding='utf-8-sig')
doc.head()

In [ ]:
import pandas as pd
PATH = "COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"
doc = pd.read_csv(PATH + "03-01-2020.csv", encoding='utf-8-sig')
doc.head()

- 3월 중순 데이터까지는 컬럼명이 Province/State, Country/Region 이고, 이후에는 Province_State, Country_Region 이므로, try except 구문을 사용해서, 데이터 조작

In [ ]:
doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
try:
    doc = doc[['Province_State', 'Country_Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
except:
    doc = doc[['Province/State', 'Country/Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
    doc.columns = ['Province_State', 'Country_Region', 'Confirmed']
    
doc.head()

### 4. 데이터프레임 데이터 변환하기
1. 특정 컬럼만 선택해서 데이터프레임 만들기
2. 특정 컬럼에 없는 데이터 삭제하기
3. 특정 컬럼의 데이터 타입 변경하기

In [ ]:
import pandas as pd
PATH = "COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"

doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
try:
    doc = doc[['Province_State', 'Country_Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
except:
    doc = doc[['Province/State', 'Country/Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
    doc.columns = ['Province_State', 'Country_Region', 'Confirmed']
doc = doc.dropna(subset=['Confirmed'])     # 2. 특정 컬럼에 없는 데이터 삭제하기
doc = doc.astype({'Confirmed': 'int64'})   # 3. 특정 컬럼의 데이터 타입 변경하기
doc.head()

- 국가 정보 가져오기

In [ ]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig')
country_info.head()

- 두 데이터프레임 합쳐보기

In [ ]:
test_df = pd.merge(doc, country_info, how='left', on='Country_Region')
test_df.head()

- 잘못 매칭된 국가 정보 확인하기
  - iso2 컬럼이 매칭되지 않은 확진자수 국가 확인해보기

In [ ]:
test_df.isnull().sum()

In [ ]:
nan_rows = test_df[test_df['iso2'].isnull()]
nan_rows.head()

### 컬럼값 변경하기
- Country_Region 국가명이 다양한 경우가 많았음
- 각 케이스를 일괄적으로 변경할 키값이 존재하지 않고, 키가 될 수 있는 컬럼도 다양하고, 각 파일마다 키가 될 수 있는 컬럼이 변경되어, 키값으로 매칭이 불가하였음
- 이에 각 케이스를 직접 확인해서, 국가명을 일관되게 변경할 수 있도록 별도 json 파일 작성
- json 파일 기반으로 국가명을 일관되게 변경하기로 함

#### json.load() 함수로 파일로된 json 데이터를 사전처럼 다룰 수 있음

In [ ]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    print (json_data.keys())

### apply() 함수 사용법
- apply() 함수를 사용해서, 특정 컬럼값 변경 가능

In [ ]:
df = pd.DataFrame({
    '영어': [60, 70],
    '수학': [100, 50]
}, index = ['Dave', 'David'])
df

In [ ]:
def func(df_data):
    print (type(df_data))    
    print (df_data.index)
    print (df_data.values)    
    return df_data

> 참고로 행이 두개 인데, 3번 func가 호출되는 이유는 apply() 함수 자체가, 첫 번째 행에 대해서는 두번 호출하도록 구현되어 있기 때문임 (전체 행의 처리를 위한 최적화 기법 적용 가능 여부를 확인코자 이와 같이 구현됨)

In [ ]:
df_func = df.apply(func, axis=0)

In [ ]:
df_func = df.apply(func, axis=1)

In [ ]:
df = pd.DataFrame({
    '영어': [60, 70],
    '수학': [100, 50]
}, index = ['Dave', 'David'])
df

In [ ]:
def func(df_data):
    df_data['영어'] = 80
    return df_data

In [ ]:
df_func = df.apply(func, axis=1)

In [ ]:
df_func

### apply() 함수를 사용해서, 국가 컬럼값 변경하기

- 사전 작업 (doc 변수로 데이터프레임 파일 만들기)

In [ ]:
import pandas as pd

doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
try:
    doc = doc[['Province_State', 'Country_Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
except:
    doc = doc[['Province/State', 'Country/Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
    doc.columns = ['Province_State', 'Country_Region', 'Confirmed']
doc = doc.dropna(subset=['Confirmed'])     # 2. 특정 컬럼에 없는 데이터 삭제하기
doc = doc.astype({'Confirmed': 'int64'})   # 3. 특정 컬럼의 데이터 타입 변경하기
doc.head()

- 변경할 국가명을 가지고 있는 json 파일 읽기

In [ ]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    print (json_data.keys())

- Country_Region 이라는 컬럼값을 확인해서, 국가명이 다르게 기재되어 있을 경우에만, 지정한 국가명으로 변경

In [ ]:
def func(row):
    if row['Country_Region'] in json_data:
        row['Country_Region'] = json_data[row['Country_Region']]
    return row

In [ ]:
doc = doc.apply(func, axis=1)
doc.head()

### 참고: 파일명으로 데이터 변환하기

- lstrip(): 앞에(왼쪽에)서 특정 데이터 삭제하기, rstrip(): 뒤에(오른쪽에)서 특정 데이터 삭제하기
- replace(변경전데이터, 변경후데이터): 문자열에서 변경전데이터 를 변경후데이터 로 변경

In [ ]:
data = "01-22-2020.csv"
date_column = data.split(".")[0].lstrip('0').replace('-', '/')
date_column

In [ ]:
data = "01-22-2020.csv"
data.split('.')[0].lstrip("0").replace('-', '/')

In [ ]:
doc.columns

In [ ]:
doc.columns = ['Province_State', 'Country_Region', date_column]
doc.columns

In [ ]:
doc.head()

### 5. 중복 데이터 합치기
- groupby() : 그룹별로 데이터를 집계하는 함수
  - 동일한 컬럼값으로 묶어서 통계 또는 평균등을 확인할 수 있음 

In [ ]:
df = pd.DataFrame({
    '성별': ['남', '남', '남'],
    '이름': ['David', 'Dave', 'Dave'],
    '수학': [100, 50, 80],
    '국어': [80, 70, 50]    
})
df

In [ ]:
# df.groupby('이름').mean() 메서드는 숫자 컬럼에 대해서만 계산 가능합니다.
# 기존에는 df.groupby('이름').mean() 호출시, 숫자 컬럼 외에는 자동 제외하고 계산하였으나, 최근 버전에서는 자동 제외되지 않으므로,
# 다음과 같이 숫자 컬럼만을 강제로 선택한 후, df.groupby('이름').mean() 을 호출하면 좋을 것 같습니다.
selected_columns = ['이름', '수학', '국어']
df = df[selected_columns]

df.groupby('이름').mean()

In [ ]:
df.groupby('이름').sum()

- 국가별 총 확진자수 구하기

In [ ]:
import pandas as pd

doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
try:
    doc = doc[['Province_State', 'Country_Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
except:
    doc = doc[['Province/State', 'Country/Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
    doc.columns = ['Province_State', 'Country_Region', 'Confirmed']
doc = doc.dropna(subset=['Confirmed'])     # 2. 특정 컬럼에 없는 데이터 삭제하기
doc = doc.astype({'Confirmed': 'int64'})   # 3. 특정 컬럼의 데이터 타입 변경하기
doc.head()

In [ ]:
doc.groupby('Country_Region').sum()  # 4. Country_Region 컬럼값이 동일한 케이스를 그룹화해서, 각 그룹별 합계 확인하기

### 6. 데이터 전처리하기
- 지금까지의 과정을 모두 한데 모아서, 함수로 만들기
  1. csv 파일 읽기
  2. 'Country_Region', 'Confirmed' 두 개의 컬럼만 가져오기
  3. 'Confirmed' 에 데이터가 없는 행 삭제하기
  4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경하기
  5. 'Confirmed' 데이터 타입을 int64(정수) 로 변경
  6. 'Country_Region' 를 기준으로 중복된 데이터를 합치기
  7. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기

In [ ]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):

    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')  # 1. csv 파일 읽기
    try:
        doc = doc[['Country_Region', 'Confirmed']]  # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
    except:
        doc = doc[['Country/Region', 'Confirmed']]  # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed'])     # 3. 특정 컬럼에 없는 데이터 삭제하기
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)   # 4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경하기
    doc = doc.astype({'Confirmed': 'int64'})   # 5. 특정 컬럼의 데이터 타입 변경하기
    doc = doc.groupby('Country_Region').sum()  # 6. 특정 컬럼으로 중복된 데이터를 합치기

    # 7. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기
    date_column = filename.split(".")[0].lstrip('0').replace('-', '/') 
    doc.columns = [date_column]
    return doc

In [ ]:
json_data

### 테스트해보기

In [ ]:
doc1 = create_dateframe("01-22-2020.csv")
doc2 = create_dateframe("04-01-2020.csv")

In [ ]:
doc2.head()

#### 데이터프레임 합치기

In [ ]:
doc = pd.merge(doc1, doc2, how='outer', left_index=True, right_index=True)
doc.head()

#### 없는 데이터는 0으로 값 대체하기

In [ ]:
doc = doc.fillna(0)
doc

#### 참고: 특정 폴더 파일 리스트 확인하기
- split() 함수를 사용해서 특정 확장자를 가진 파일 리스트만 추출 가능
- 문자열변수.split('.') 은 ['파일명', '확장자'] 와 같은 리스트가 반환되므로, 문자열변수.split('.')[-1] 을 통해, 이 중에서 마지막 아이템을 선택하면 됨
  

In [ ]:
import os

PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
file_list = os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)

print (csv_list)

#### 참고: 리스트 정렬
- 리스트변수.sort() : 오름차순 정렬 (디폴트)
- 리스트변수.sort(reverse=True) : 내림차순 정렬

In [ ]:
csv_list.sort()
csv_list

### 6. 여러 데이터 수집, 전처리해서, 하나의 데이터프레임 만들기
- 지금까지의 과정을 모두 한데 모아서, 함수로 만들기
  1. 필요한 파일 리스트만 추출하기
  2. 파일 리스트 정렬하기
  3. 데이터프레임 전처리하기 (별도 create_dateframe() 함수)
  4. 데이터프레임 합치기

#### 최종 코드

In [ ]:
import json
import pandas as pd

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):

    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')  # 1. csv 파일 읽기
    try:
        doc = doc[['Country_Region', 'Confirmed']]  # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
    except:
        doc = doc[['Country/Region', 'Confirmed']]  # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed'])     # 3. 특정 컬럼에 없는 데이터 삭제하기
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)   # 4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경하기
    doc = doc.astype({'Confirmed': 'int64'})   # 5. 특정 컬럼의 데이터 타입 변경하기
    doc = doc.groupby('Country_Region').sum()  # 6. 특정 컬럼으로 중복된 데이터를 합치기

    # 7. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기
    date_column = filename.split(".")[0].lstrip('0').replace('-', '/') 
    doc.columns = [date_column]
    return doc

In [ ]:
import os

def generate_dateframe_by_path(PATH):

    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split(".")[-1] == 'csv':
            csv_list.append(file)
    csv_list.sort()
    
    for file in csv_list:
        doc = create_dateframe(file)
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)

    final_doc = final_doc.fillna(0)
    return final_doc

In [ ]:
PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
doc = generate_dateframe_by_path(PATH)
doc

#### 참고: 데이터 타입 변환이 가능한 모든 열의 데이터 타입 변경
- pd.astype(데이터타입)
  - object 는 파이썬의 str 또는 혼용 데이터 타입 (문자열)
  - int64 는 파이썬의 int (정수)
  - float64 는 파이썬의 float (부동소숫점)
  - bool 는 파이썬의 bool (True 또는 False 값을 가지는 boolean)

In [ ]:
doc = doc.astype('int64')
doc

#### pandas 라이브러리로 csv 파일 쓰기
- pandas dataframe 데이터를 csv 파일로 저장하기 위해, to_csv() 함수 사용
    ```
    doc.to_csv("00_data/students_default.csv")
    ```

- encoding 옵션 사용 가능
    ```
    doc.to_csv("00_data/students_default.csv", encoding='utf-8-sig')
    ```

In [ ]:
doc.to_csv("COVID-19-master/final_df.csv")

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>